In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import svd
from scipy.sparse.linalg import svds

# 불러올 파일
# 1. 5000명 사용자-강의실 점수 TL.csv 
# 2. 강의실 건물별 TOP3 최종.csv
# 3. 건물별 이동시간 최종.csv

score_df = pd.read_csv('/content/5000명 사용자-강의실 점수 TL.csv')
classroom_df = pd.read_csv('/content/강의실 건물별 TOP3 최종.csv')
building_distance = pd.read_csv('/content/건물별 이동시간 최종.csv')

# 강의실에 대한 사용자 점수(true label) 불러오기
score_df.score = score_df.score.apply(eval)
score_df.score = score_df.score.apply(np.array)


############ 챗봇으로 받은 사용자의 score 예측 ##############
#############################################################
def user_score(num_user, time, location, plug, table, chair):

  # 사용자 선택 랜드마크와 강의실 사이 거리구하기
  user_building = location
  distance = building_distance[['building', user_building]].copy()
  distance.rename(columns = {user_building: 'distance'}, inplace = True)
  classroom_with_dis = pd.merge(classroom_df, distance, on= 'building', how ='left')


  # 사용자 선호도 의자, 책상 만족시키는 강의실 찾기
  satisfy_user_idx = classroom_with_dis['table'] == table
  desk_score = classroom_with_dis['table'].copy()
  desk_score[satisfy_user_idx] == 1
  desk_score[~satisfy_user_idx] == 0

  satisfy_user_idx = classroom_with_dis['chair'] == chair
  chair_score = classroom_with_dis['chair'].copy()
  chair_score[satisfy_user_idx] == 1
  chair_score[~satisfy_user_idx] == 0


  # max min Normalization 사용 : (x-min(x) /(max(x)-min(x))
  plug_norm = ((classroom_with_dis['plugs']*plug) - (classroom_with_dis['plugs']*plug).min()) / ((classroom_with_dis['plugs']*plug).max()-(classroom_with_dis['plugs']*plug).min())
  distance_norm = (classroom_with_dis['distance'] - classroom_with_dis['distance'].min() ) / (classroom_with_dis['distance'].max() - classroom_with_dis['distance'].min())
  size_norm = (classroom_with_dis['max'] - classroom_with_dis['max'].min() ) / (classroom_with_dis['max'].max() - classroom_with_dis['max'].min())
  ac_norm = (classroom_with_dis['ac_bool'] - classroom_with_dis['ac_bool'].min() ) / (classroom_with_dis['ac_bool'].max() - classroom_with_dis['ac_bool'].min())
  desk_norm = (desk_score - desk_score.min() ) / (desk_score.max() - desk_score.min())
  chair_norm = (chair_score - chair_score.min() ) / (chair_score.max() - chair_score.min())

  # 사용자에 대한 선호도 norm DataFrame 생성 
  norm_col = {
        'classroom_index':classroom_with_dis['classroom_index'],
        'plug_norm':plug_norm*0.86,
        'distance_norm':distance_norm*(-5), 
        'size_norm': size_norm*1,
        'ac_norm':ac_norm*0.6,
        'table_norm' : desk_norm * 0.68,    
        'chair_norm' : chair_norm * 0.68
        }
  mmnorm = pd.DataFrame(norm_col)

  # 사용자 입장에서 각 강의실 점수 계산 + sort
  score = pd.concat([mmnorm['classroom_index'], mmnorm[['plug_norm','distance_norm','size_norm','ac_norm', 'table_norm', 'chair_norm']].sum(axis=1)], axis =1)
  score.columns=['classroom_index','score']

  # 사용자 점수 배열 생성
  classroom_score = score['score'].copy().tolist()
  return classroom_score
#############################################################


################ 강의실 9개 추천 ############################
#############################################################
def classroom_Top9(num_user, time, location, plug, table, chair):
  input_user_score = user_score(num_user, time, location, plug, table, chair)

  # matrix로 변환
  user_score_matrix = np.array(list(score_df['score']))
  user_score_matrix = np.concatenate((user_score_matrix, np.array(input_user_score).reshape((1,82))), axis=0)

  # 평균을 이용한 유저별 중심화
  user_score_mean = np.mean(user_score_matrix, axis=1)
  user_score_matrix_centered = user_score_matrix - user_score_mean.reshape(-1, 1)
  user_score_matrix_centered

  # Singular Value Decomposition
  # k는 latent factor의 개수. 개수 조정하면 다른 결과 얻을 수 있음.
  # U, s, Vt = svd(user_score_matrix_centered, full_matrices=False)
  U, s, Vt = svds(user_score_matrix_centered, k=3)
  S = np.diag(s)

  # SVD로 각 사용자별 점수 예측
  predicted_scores = np.dot(np.dot(U, S), Vt) + user_score_mean.reshape(-1, 1)
  predicted_scores[0]

  # 예측 행렬에서 사용자 index에 대해 강의실 index 9개 추천해주는 함수
  def make_recommendation(original, svd_prediction, user_index, num_recommendations=9):
    score = svd_prediction[user_index]
    rec_inx = score.argsort()[-num_recommendations:][::-1]
    rec_val = svd_prediction[user_index][rec_inx]
    return rec_inx, rec_val

  # rec_inx가 강의실 index
  which_user = -1
  rec_inx, rec_val = make_recommendation(user_score_matrix_centered, predicted_scores, which_user, 9)
  rec_inx

  # 추천할 9개의 강의실 최종 선택
  top9_df = pd.DataFrame(rec_inx, columns =['classroom_index'])
  top9 = pd.merge(top9_df, classroom_df, on= 'classroom_index', how ='left')
  output = top9[['building', 'room']].copy().to_records(index=False)

  return output
#############################################################



# 챗봇 input 넣는 곳
result = classroom_Top9(None, None, '국제관', 2, 1, 1)
print(result)


[('L-P', '436(SUPEX HALL)') ('법학관(구관)', 'B101') ('법학관(구관)', '207')
 ('현대자동차경영관', 'B201') ('현대자동차경영관', 'B202') ('L-P', '108')
 ('사범대학신관', '403') ('사범대학신관', '232') ('L-P', '205')]
